# Adding paths and files

In [3]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), "..", "..", ".."))
from configs import spark_config as config
from utils import spark_utils as utils

# Specify the directory where your CSV files are located
directory = os.path.join(config.RAW_DATA_DIR, "t20s_csv2")
print(directory)

client = utils.get_hdfs_client()

all_files = client.list(directory)
info_files = [os.path.join(directory, file) for file in all_files if "info" in file]

matches = []
# Print the list of CSV files
for info_file in info_files:
    matches.append(info_file.split("/")[-1])

match_ids = []
for csv_file in matches:
    match_ids.append(csv_file.split("_")[0])

/usr/ravi/t20/data/1_rawData/t20s_csv2
[2024-11-24T13:53:58.395+0530] {base.py:84} INFO - Retrieving connection 'webhdfs_default'
[2024-11-24T13:53:58.397+0530] {webhdfs.py:82} INFO - Trying to connect to 192.168.245.142:9870
[2024-11-24T13:53:58.398+0530] {webhdfs.py:86} INFO - Trying namenode 192.168.245.142
[2024-11-24T13:53:58.401+0530] {client.py:192} INFO - Instantiated <InsecureClient(url='http://192.168.245.142:9870/')>.
[2024-11-24T13:53:58.403+0530] {client.py:320} INFO - Fetching status for '/'.
[2024-11-24T13:53:58.433+0530] {webhdfs.py:96} INFO - Using namenode 192.168.245.142 for hook
[2024-11-24T13:53:58.435+0530] {client.py:1116} INFO - Listing '/usr/ravi/t20/data/1_rawData/t20s_csv2'.


In [4]:
import pandas as pd
import io


def process_single_match(client, match_id: str) -> pd.DataFrame:
    """
    Process a single match file and return the processed DataFrame or None on failure.

    Args:
        client: The HDFS client for file operations.
        match_id: Unique ID for each match.

    Returns:
        DataFrame of the processed match or None if an error occurs.
    """
    try:
        with client.read(
            os.path.join(config.RAW_DATA_DIR, "t20s_csv2", f"{match_id}_info.csv")
        ) as reader:
            data = reader.read()
        match_df = pd.read_csv(
            io.StringIO(data.decode("utf-8")),
            header=None,
            names=["col1", "attributes", "values", "players", "code"],
        )
        match_df = match_df.drop(columns=["col1", "players", "code"]).T
        match_df.columns = match_df.iloc[0]
        match_df["match_id"] = match_id
        match_df = match_df[
            ["match_id", "team", "team", "gender", "season", "winner"]
        ].drop("attributes")
        match_df = match_df.reset_index(drop=True)
        return match_df
    except Exception as e:
        print(f"Error processing match {match_id}: {e}")
        return None

In [5]:
from tqdm import tqdm
import concurrent.futures
import logging

logging.basicConfig(level=logging.ERROR)
recalculated_matches = []

# Set the logging level for the HDFS client to WARNING
logging.getLogger("hdfs.client").setLevel(logging.CRITICAL)
injured_matches = []

# Process each match concurrently using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {
        executor.submit(process_single_match, client, match_id): match_id
        for match_id in match_ids
    }

    # Gather results with tqdm for progress tracking
    for future in tqdm(
        concurrent.futures.as_completed(futures),
        total=len(futures),
        desc="Processing Matches",
    ):
        match = futures[future]
        try:
            result = future.result()
            if result is not None:
                recalculated_matches.append(result)
            else:
                injured_matches.append(match)
        except Exception as e:
            injured_matches.append(match)

if not recalculated_matches:
    print("No matches were successfully processed.")
    raise ValueError("No matches were successfully processed.")

matches_data = pd.concat(recalculated_matches, ignore_index=True)
matches_data

[2024-11-24T13:54:48.346+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: 192.168.245.142. Connection pool size: 10
[2024-11-24T13:54:48.349+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: 192.168.245.142. Connection pool size: 10
[2024-11-24T13:54:48.350+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: 192.168.245.142. Connection pool size: 10
[2024-11-24T13:54:48.359+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: 192.168.245.142. Connection pool size: 10
[2024-11-24T13:54:48.666+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: 192.168.245.142. Connection pool size: 10


Processing Matches:   0%|          | 0/3825 [00:00<?, ?it/s]

[2024-11-24T13:54:48.688+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: 192.168.245.142. Connection pool size: 10
[2024-11-24T13:54:49.340+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: ravikumar.. Connection pool size: 10
[2024-11-24T13:54:49.352+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: ravikumar.. Connection pool size: 10
[2024-11-24T13:54:49.364+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: ravikumar.. Connection pool size: 10
[2024-11-24T13:54:49.405+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: ravikumar.. Connection pool size: 10


Processing Matches:   0%|          | 16/3825 [00:00<01:24, 45.27it/s]

Error processing match 1041617: "['winner'] not in index"
[2024-11-24T13:54:49.845+0530] {connectionpool.py:329} WARNING - Connection pool is full, discarding connection: ravikumar.. Connection pool size: 10


...

Error processing match 902653: "['winner'] not in index"


Processing Matches:  98%|█████████▊| 3755/3825 [00:52<00:00, 73.47it/s]

Error processing match 951319: "['winner'] not in index"


Processing Matches: 100%|██████████| 3825/3825 [00:53<00:00, 71.53it/s]


attributes,match_id,team,team,team,team,gender,season,winner
0,1019981,New Zealand,Bangladesh,New Zealand,Bangladesh,male,2016/17,New Zealand
1,1031665,West Indies,England,West Indies,England,male,2017,West Indies
2,1001349,Australia,Sri Lanka,Australia,Sri Lanka,male,2016/17,Sri Lanka
3,1034825,India,England,India,England,male,2016/17,England
4,1019979,New Zealand,Bangladesh,New Zealand,Bangladesh,male,2016/17,New Zealand
...,...,...,...,...,...,...,...,...
3705,966761,India,United Arab Emirates,India,United Arab Emirates,male,2015/16,India
3706,966751,India,Pakistan,India,Pakistan,male,2015/16,India
3707,995469,Sri Lanka,Australia,Sri Lanka,Australia,male,2016,Australia
3708,995467,Sri Lanka,Australia,Sri Lanka,Australia,male,2016,Australia


In [8]:
if recalculated_matches:
    # Concatenate all match DataFrames
    matches_data = pd.concat(recalculated_matches, ignore_index=True)
    matches_data.columns = [
        "match_id",
        "team1",
        "team2",
        "team1_duplicate",
        "team2_duplicate",
        "gender",
        "season",
        "winner",
    ]
    matches_data = matches_data.drop(columns=["team1_duplicate", "team2_duplicate"])

    # Data quality checks
    if matches_data.empty:
        print("No match data consolidated.")
        raise ValueError("Consolidated match data is empty.")
    required_columns = ["match_id", "team1", "team2", "gender", "season", "winner"]
    missing_columns = [
        col for col in required_columns if col not in matches_data.columns
    ]
    if missing_columns:
        print(f"Missing columns in matches data: {missing_columns}")
        raise ValueError(f"Missing columns in matches data: {missing_columns}")

    # Save matches_data directly to HDFS
    utils.ensure_hdfs_directory(client, config.PROCESSED_DATA_DIR)
    matches_csv_path = os.path.join(config.PROCESSED_DATA_DIR, "matches.csv")
    csv_data = matches_data.to_csv(index=False)
    client.write(matches_csv_path, data=csv_data, overwrite=True)

    print(f"Successfully processed matches: {len(recalculated_matches)}")
    print(f"Failed matches: {len(injured_matches)}")
    print("Matches data processing and saving completed successfully.")
else:
    print("No matches were successfully processed.")
    raise Exception("No matches were successfully processed.")

Successfully processed matches: 3710
Failed matches: 115
Matches data processing and saving completed successfully.
